In [3]:
import matplotlib.pyplot as plt
import IPython.display as ipd
import numpy as np
import sys
import time
from itertools import combinations
from functools import reduce
import constraint as c 

# Allows me to import my modules
sys.path.append('./modules')
from audio_utils import *

Attempting to read settings file...
	Read successfully!


In [28]:
augmented_fd = get_file_classes("pre_multiclass_augmented")

In [29]:
cropped_fd = get_file_classes("cropped")

In [30]:
print("cropped amount:", len(cropped_fd), "\tpre_auged amount:", len(augmented_fd))

cropped amount: 922 	pre_auged amount: 13640


In [5]:
cropped_fd[0]

{'augmentations': {},
 'filepath': 'audio_data\\cropped_data\\beater\\bass_drum\\normal\\0.gz',
 'labels': {'hit_label': ['beater'],
  'kit_label': ['bass_drum'],
  'tech_label': ['normal']}}

In [6]:
augmented_fd[0]

{'augmentations': {'pre_augs': ['amplitude']},
 'filepath': 'audio_data\\pre-multiclass_augmented_data\\amplitude\\beater\\bass_drum\\normal\\0.gz',
 'labels': {'hit_label': ['beater'],
  'kit_label': ['bass_drum'],
  'tech_label': ['normal']}}

In [7]:
sample_data = {kit_label: [] for kit_label in SETTINGS.label["kit_label"]}
for fd in cropped_fd+augmented_fd:
    for kl in sample_data.keys():
        if (kl in fd["labels"]["kit_label"]):
            sample_data[kl].append(fd)

In [8]:
sample_data["bass_drum"][:2]

[{'augmentations': {},
  'filepath': 'audio_data\\cropped_data\\beater\\bass_drum\\normal\\0.gz',
  'labels': {'hit_label': ['beater'],
   'kit_label': ['bass_drum'],
   'tech_label': ['normal']}},
 {'augmentations': {},
  'filepath': 'audio_data\\cropped_data\\beater\\bass_drum\\normal\\1.gz',
  'labels': {'hit_label': ['beater'],
   'kit_label': ['bass_drum'],
   'tech_label': ['normal']}}]

In [9]:
samples = np.zeros(12000, dtype=np.float)

In [4]:
hit_breakdown = {hl: [] for hl in SETTINGS.label["hit_label"]}
for hl in SETTINGS.label["hit_label"]:
    for kl in SETTINGS.hierarchy[hl].keys():
        for tl in SETTINGS.hierarchy[hl][kl]:
            hit_breakdown[hl].append("-".join([kl, tl]))

In [5]:
hit_breakdown

{'beater': ['bass_drum-normal'],
 'stick': ['hi_hat-normal',
  'hi_hat-open',
  'ride-normal',
  'ride-bell',
  'high_tom-normal',
  'mid_tom-normal',
  'crash-normal',
  'low_tom-normal',
  'snare-normal',
  'splash-normal']}

In [45]:
def unique_kit_class(*ts):
    current_kt = ts[0].split("-")[0]
    for t in ts[1:]:
        kt = t.split("-")[0]
        if kt == current_kt:
            return False
    return True

In [7]:
prob = c.Problem()
prob.addVariable("beater", hit_breakdown["beater"])
prob.getSolutions()

[{'beater': 'bass_drum-normal'}]

In [8]:
prob = c.Problem()
prob.addVariable("stick", hit_breakdown["stick"])
prob.getSolutions()

[{'stick': 'splash-normal'},
 {'stick': 'snare-normal'},
 {'stick': 'low_tom-normal'},
 {'stick': 'crash-normal'},
 {'stick': 'mid_tom-normal'},
 {'stick': 'high_tom-normal'},
 {'stick': 'ride-bell'},
 {'stick': 'ride-normal'},
 {'stick': 'hi_hat-open'},
 {'stick': 'hi_hat-normal'}]

In [24]:
prob = c.Problem()
prob.addVariable("beater", hit_breakdown["beater"])
prob.addVariable("stick", hit_breakdown["stick"])
prob.getSolutions()

[{'beater': 'bass_drum-normal', 'stick': 'splash-normal'},
 {'beater': 'bass_drum-normal', 'stick': 'snare-normal'},
 {'beater': 'bass_drum-normal', 'stick': 'low_tom-normal'},
 {'beater': 'bass_drum-normal', 'stick': 'crash-normal'},
 {'beater': 'bass_drum-normal', 'stick': 'mid_tom-normal'},
 {'beater': 'bass_drum-normal', 'stick': 'high_tom-normal'},
 {'beater': 'bass_drum-normal', 'stick': 'ride-bell'},
 {'beater': 'bass_drum-normal', 'stick': 'ride-normal'},
 {'beater': 'bass_drum-normal', 'stick': 'hi_hat-open'},
 {'beater': 'bass_drum-normal', 'stick': 'hi_hat-normal'}]

In [23]:
prob = c.Problem()
prob.addVariables(["stickA", "stickB"], hit_breakdown["stick"])
# Kit type contraint (they must be unique)
prob.addConstraint(unique_kit_class)
double_stick_combo = list(reduce(lambda acc, x: [*acc, set(x.values())] if set(x.values()) not in acc else acc, prob.getSolutions(), []))
print(len(double_stick_combo), double_stick_combo)

43 [{'splash-normal', 'snare-normal'}, {'splash-normal', 'low_tom-normal'}, {'splash-normal', 'crash-normal'}, {'splash-normal', 'mid_tom-normal'}, {'splash-normal', 'high_tom-normal'}, {'splash-normal', 'ride-bell'}, {'splash-normal', 'ride-normal'}, {'splash-normal', 'hi_hat-open'}, {'splash-normal', 'hi_hat-normal'}, {'low_tom-normal', 'snare-normal'}, {'crash-normal', 'snare-normal'}, {'snare-normal', 'mid_tom-normal'}, {'high_tom-normal', 'snare-normal'}, {'snare-normal', 'ride-bell'}, {'ride-normal', 'snare-normal'}, {'snare-normal', 'hi_hat-open'}, {'hi_hat-normal', 'snare-normal'}, {'low_tom-normal', 'crash-normal'}, {'low_tom-normal', 'mid_tom-normal'}, {'low_tom-normal', 'high_tom-normal'}, {'low_tom-normal', 'ride-bell'}, {'low_tom-normal', 'ride-normal'}, {'low_tom-normal', 'hi_hat-open'}, {'low_tom-normal', 'hi_hat-normal'}, {'crash-normal', 'mid_tom-normal'}, {'high_tom-normal', 'crash-normal'}, {'crash-normal', 'ride-bell'}, {'ride-normal', 'crash-normal'}, {'crash-norma

In [25]:
prob = c.Problem()
prob.addVariable("beater", hit_breakdown["beater"])
prob.addVariables(["stickA", "stickB"], hit_breakdown["stick"])
# Kit type contraint (they must be unique)
prob.addConstraint(unique_kit_class)
double_stick_combo = list(reduce(lambda acc, x: [*acc, set(x.values())] if set(x.values()) not in acc else acc, prob.getSolutions(), []))
print(len(double_stick_combo), double_stick_combo)

43 [{'bass_drum-normal', 'snare-normal', 'splash-normal'}, {'bass_drum-normal', 'low_tom-normal', 'splash-normal'}, {'bass_drum-normal', 'crash-normal', 'splash-normal'}, {'bass_drum-normal', 'mid_tom-normal', 'splash-normal'}, {'bass_drum-normal', 'high_tom-normal', 'splash-normal'}, {'bass_drum-normal', 'splash-normal', 'ride-bell'}, {'bass_drum-normal', 'ride-normal', 'splash-normal'}, {'bass_drum-normal', 'splash-normal', 'hi_hat-open'}, {'bass_drum-normal', 'hi_hat-normal', 'splash-normal'}, {'bass_drum-normal', 'low_tom-normal', 'snare-normal'}, {'bass_drum-normal', 'crash-normal', 'snare-normal'}, {'bass_drum-normal', 'snare-normal', 'mid_tom-normal'}, {'bass_drum-normal', 'high_tom-normal', 'snare-normal'}, {'bass_drum-normal', 'snare-normal', 'ride-bell'}, {'bass_drum-normal', 'ride-normal', 'snare-normal'}, {'bass_drum-normal', 'snare-normal', 'hi_hat-open'}, {'bass_drum-normal', 'hi_hat-normal', 'snare-normal'}, {'bass_drum-normal', 'low_tom-normal', 'crash-normal'}, {'bass_

In [48]:
def kit_combinations(fd):
    hit_combos = SETTINGS.data["multiclassed"]["hit_combinations"]
    hit_breakdown = {hl: [] for hl in SETTINGS.label["hit_label"]}
    for hl in SETTINGS.label["hit_label"]:
        for kl in SETTINGS.hierarchy[hl].keys():
            for tl in SETTINGS.hierarchy[hl][kl]:
                hit_breakdown[hl].append("-".join([kl, tl]))
    kit_combos = []
    for hc in hit_combos:
        prob = c.Problem()
        var_i = 0
        for ht in hc:
            prob.addVariable(str(var_i), hit_breakdown[ht])
            var_i += 1
        # Kit type contraint (they must be unique)
        prob.addConstraint(unique_kit_class)
        # Remove permutations of hits
        kit_combo_subset = list(reduce(lambda acc, x: [*acc, set(x.values())] if set(x.values()) not in acc else acc, prob.getSolutions(), []))
        for subset in kit_combo_subset:
            kit_combos.append(subset)
    return kit_combos

In [49]:
kcs = kit_combinations(cropped_fd)

In [51]:
kttt_count = {}
for kttt in hit_breakdown["beater"]+hit_breakdown["stick"]:
    count = 0
    for kc in kcs:
        if kttt in kc:
            count += 1
    kttt_count[kttt] = count

In [52]:
kttt_count

{'bass_drum-normal': 54,
 'crash-normal': 20,
 'hi_hat-normal': 18,
 'hi_hat-open': 18,
 'high_tom-normal': 20,
 'low_tom-normal': 20,
 'mid_tom-normal': 20,
 'ride-bell': 18,
 'ride-normal': 18,
 'snare-normal': 20,
 'splash-normal': 20}

In [54]:
print(len(kcs))
kcs

107


[{'bass_drum-normal'},
 {'splash-normal'},
 {'snare-normal'},
 {'low_tom-normal'},
 {'crash-normal'},
 {'mid_tom-normal'},
 {'high_tom-normal'},
 {'ride-bell'},
 {'ride-normal'},
 {'hi_hat-open'},
 {'hi_hat-normal'},
 {'bass_drum-normal', 'splash-normal'},
 {'bass_drum-normal', 'snare-normal'},
 {'bass_drum-normal', 'low_tom-normal'},
 {'bass_drum-normal', 'crash-normal'},
 {'bass_drum-normal', 'mid_tom-normal'},
 {'bass_drum-normal', 'high_tom-normal'},
 {'bass_drum-normal', 'ride-bell'},
 {'bass_drum-normal', 'ride-normal'},
 {'bass_drum-normal', 'hi_hat-open'},
 {'bass_drum-normal', 'hi_hat-normal'},
 {'snare-normal', 'splash-normal'},
 {'low_tom-normal', 'splash-normal'},
 {'crash-normal', 'splash-normal'},
 {'mid_tom-normal', 'splash-normal'},
 {'high_tom-normal', 'splash-normal'},
 {'ride-bell', 'splash-normal'},
 {'ride-normal', 'splash-normal'},
 {'hi_hat-open', 'splash-normal'},
 {'hi_hat-normal', 'splash-normal'},
 {'low_tom-normal', 'snare-normal'},
 {'crash-normal', 'snare-